This notebook runs from the Anaconda distribution and has the "findspark" Python Module installed.
Refer to https://www.learntospark.com/2019/12/install-spark-in-windows-using-jupyter-notebook.html

In [1]:
import findspark
findspark.init()

In [2]:
findspark.find()

'/usr/local/opt/apache-spark/libexec'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
# Build a SparkSession, using the SparkSession APIs.
# If one does not exist, then create an instance. There
# can only be one SparkSession per JVM
spark = (SparkSession
    .builder
    .appName("TestNotebook")
    .getOrCreate())

22/04/02 22:12:17 WARN Utils: Your hostname, Stephens-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.17 instead (on interface en0)
22/04/02 22:12:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/02 22:12:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
mnm_file = "/Users/stephenmoss/Documents/Development/Spark/learning-spark-v2/ch02/data/mnm_dataset.csv"

# Read the file into a Spark DataFrame using the CSV
# format by inferring the schema and specifying that the
# file contains a header, which contains column names for
# comma-separated fields
mnm_df = (spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(mnm_file))

In [9]:
# We use DataFrame high-level APIs. Note
# that we don't use RDDs at all. Because some of Spark's
# functions reeturn the same object we can chain function calls.
# 1. Select from the DataFrame the fields "State", "Color", and "Count"
# 2. Since we want to group each state and its M&M color count,
#    we use groupBy()
# 3. Aggregate counts of all colors and groupBy() State and Color
# 4. orderBy() in descending order
count_mnm_df = (mnm_df
    .select("State", "Color", "Count")
    .groupBy("State", "Color")
    .agg(count("Count").alias("Total"))
    .orderBy("Total", ascending=False))
    
# Show the resulting aggregations for all the states and colors;
# a total count of each color per state
# Note show() is an action, which will trigger the above
# query to be executed
count_mnm_df.show(n=60, truncate=False)
print("Total Rows = %d" % (count_mnm_df.count()))

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|WA   |Green |1779 |
|OR   |Orange|1743 |
|TX   |Green |1737 |
|TX   |Red   |1725 |
|CA   |Green |1723 |
|CO   |Yellow|1721 |
|CA   |Brown |1718 |
|CO   |Green |1713 |
|NV   |Orange|1712 |
|TX   |Yellow|1703 |
|NV   |Green |1698 |
|AZ   |Brown |1698 |
|WY   |Green |1695 |
|CO   |Blue  |1695 |
|NM   |Red   |1690 |
|AZ   |Orange|1689 |
|NM   |Yellow|1688 |
|NM   |Brown |1687 |
|UT   |Orange|1684 |
|NM   |Green |1682 |
|UT   |Red   |1680 |
|AZ   |Green |1676 |
|NV   |Yellow|1675 |
|NV   |Blue  |1673 |
|WA   |Red   |1671 |
|WY   |Red   |1670 |
|WA   |Brown |1669 |
|NM   |Orange|1665 |
|WY   |Blue  |1664 |
|WA   |Yellow|1663 |
|WA   |Orange|1658 |
|CA   |Orange|1657 |
|NV   |Brown |1657 |
|CA   |Red   |1656 |
|CO   |Brown |1656 |
|UT   |Blue  |1655 |
|AZ   |Yellow|1654 |
|TX   |Orange|1652 |
|AZ   |Red   |1648 |
|OR   |Blue  |1646 |
|UT   |Yellow|1645 |
|OR   |Red   |1645 |
|CO   |Orange|1642 |
|TX   |Brown 

Total Rows = 60


In [10]:
# While the above code aggregated and counted for all
# the states, what if we want to see the data for a single
# state, e.g. CA?
# 1. Select from all rows in the DataFrame
# 2. Filter only CA state
# 3. groupBy() State and Color as we did above
# 4. Aggregate the counts for each color
# 5. orderBy() in descending order
# 6. Find the aggregate count for California by filtering
ca_count_mnm_df = (mnm_df
    .select("State", "Color", "Count")
    .where(mnm_df.State == "CA")
    .groupBy("State", "Color")
    .agg(count("Count").alias("Total"))
    .orderBy("Total", ascending=False))

# Show the resulting aggregation for California
# As above, show() is an action that will trigger
# the execution of the entire computation
ca_count_mnm_df.show(n=10, truncate=False)

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|CA   |Green |1723 |
|CA   |Brown |1718 |
|CA   |Orange|1657 |
|CA   |Red   |1656 |
|CA   |Blue  |1603 |
+-----+------+-----+

